# Classify the GeoTif data from WorldClim

## Initialisation and setup

Imports

In [ ]:
# Some print utilities to make output more digestible
from utilities.printUtils import cPrint, hPrint
from utilities.printUtils import AnsiColours as Cols

Initialise the constants needed to perform the analysis.

In [ ]:
REQUIRED_VARS = ["bio", "tavg", "prec"]
CHUNKS = 100
NUM_THREADS = 3